Labels will be from 8 classes:<br>
Locality (L) <br>
Total Price (P) <br>
Land Area (LA) <br>
Cost per land area (C) <br> 
Contact name (N) <br>
Contact telephone (T) <br> 
Attributes of the property (A) <br>
Other (O)

In [4]:
import nltk
import sklearn_crfsuite
import eli5

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import numpy as np

In [5]:
def greater(word,lim):
    count=0
    for i in word:
        if(i.isdigit()):
            count+=1
    return count>=lim

In [6]:
greater("anke12",3)

False

In [7]:
def read_data(data_path,label=True):
    x=[]
    y=[]
    with open(data_path,'r') as f:
        tokens=[]
        labels=[]
        count=0
        for token in f:
            count+=1
            if(count%10000==0):
                print(count)
            token=token.strip().split(" ")
            if(len(token)==1 and token[0]==''):
                x.append(list(tokens))
                y.append(list(labels))
                tokens=[]
                labels=[]
                
            else:
                tokens.append(token[0])
                labels.append(token[1])
    return x,y

def get_pos(x):
    new_x=[]
    for sent in x:
        new_x.append(nltk.pos_tag(sent))
    return new_x

In [9]:
x_train,y_train=read_data("crf_train.txt")
x_val,y_val=read_data("crf_train.txt")
x_test,y_test=read_data("crf_train.txt")

10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [10]:
train_pos=get_pos(x_train)
val_pos=get_pos(x_val)
test_pos=get_pos(x_test)

In [11]:
print(train_pos[0:5])

[[('amarpali', 'NN'), ('#EdenPark', 'NN'), ('1625', 'CD'), ('ft', 'NN'), ('4th', 'CD'), ('felor', 'NN'), ('3', 'CD'), ('bhk', 'NN'), ('#Sector50', 'NNP'), ('rent', 'NN'), ('25000', 'CD'), ('contact', 'NN'), ('9911988332', 'CD')], [('Bhatia', 'NNP'), ('prop', 'NN'), ('.', '.'), ('#ParasTierea', 'JJ'), ('Avl', 'NNP'), ('.', '.'), ('825', 'CD'), ('sq', 'NN'), ('.', '.'), ('ft', 'NN'), ('.', '.'), ('GF', 'NNP'), ('for', 'IN'), ('sale', 'NN'), ('@5000', 'CD'), ('/-', 'JJ'), ('#ParasTierea', 'NN'), ('-137', 'NN')], [('Available', 'JJ'), ('//', 'NNP'), ('Sell', 'NNP'), ('//', 'NNP'), ('Residential', 'NNP'), ('ats', 'NNS'), ('village', 'NN')], [('+919999381560', 'NN'), (':', ':'), ('READY', 'JJ'), ('TO', 'NNP'), ('MOVE', 'NNP'), ('IN', 'NNP'), ('PROPERTY', 'NNP'), ('(', '('), ('fresh', 'JJ'), ('deal', 'NN'), ('):', 'NNP'), ('#GardeniaGolfCity', 'NN'), (',', ','), ('#Sector75', 'NNP'), (',', ','), ('NOIDA', 'NNP'), (',', ','), ('2BHK', 'CD'), (':', ':'), ('950', 'CD'), ('SQFT', 'NNP'), (',', ',

In [12]:
def hassec(word):
    return 'sec' in word.lower()

In [13]:
def hashttp(word):
    return 'http' in word.lower()

In [14]:
def hasgen(word,st):
    return st in word.lower()

In [15]:
def hasPfeat(word):
    word=word.lower()
    if 'lac' in word or 'cr ' in word or 'rs' in word or 'lakh' in word or 'crore' in word:
        return True
    return False
    

In [16]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[-4:]': word[-4:],        
        'word[:2]': word[:2],        
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'digit>=8': greater(word,8),
        'digit>=3': greater(word,3),
        'hassec': hassec(word),
        'hashttp': hashttp(word),
        'hasPfeat': hasPfeat(word),
        'hasbhk': hasgen(word,'bhk'),
        'hassq': hasgen(word,'sq'),
        'has/': hasgen(word,'/'),
        'has@': hasgen(word,'@'),
        'per': hasgen(word,'per'),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
#             '-1word[-3:]': word1[-3:],
#             '-1word[-2:]': word1[-2:],
#             '-1word[:2]': word1[:2],        
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1digit>=8': greater(word1,8),
            '-1digit>=3': greater(word1,3),            
            '-1hassec': hassec(word1),
            '-1hashttp': hashttp(word1),
            '-1hasPfeat': hasPfeat(word1),
            '-1hasbhk': hasgen(word1,'bhk'),
            '-1hassq': hasgen(word1,'sq'),        
            '-1has/': hasgen(word1,'/'),
            '-1has@': hasgen(word1,'@'),
            '-1per': hasgen(word1,'per') or ((i>1)and hasgen(sent[i-2][0],'per')),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
#             '+1word[-3:]': word1[-3:],
#             '+1word[-2:]': word1[-2:],
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1digit>=8': greater(word1,8),
            '+1digit>=3': greater(word1,3),
            '+1hassec': hassec(word1),
            '+1hashttp': hashttp(word1),
            '+1hasPfeat': hasPfeat(word1),
            '+1hasbhk': hasgen(word1,'bhk'),
            '+1hassq': hasgen(word1,'sq'),        
            '+1has/': hasgen(word1,'/'),
            '+1has@': hasgen(word1,'@'),
            '+1per': hasgen(word1,'per') or ((i<len(sent)-2)and hasgen(sent[i+2][0],'per')),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]


In [17]:
train_feat=[sent2features(x) for x in train_pos]
val_feat=[sent2features(x) for x in val_pos]
test_feat=[sent2features(x) for x in test_pos]

In [18]:
train_feat[0]

[{'bias': 1.0,
  'word.lower()': 'amarpali',
  'word[-3:]': 'ali',
  'word[-2:]': 'li',
  'word[-4:]': 'pali',
  'word[:2]': 'am',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'NN',
  'postag[:2]': 'NN',
  'digit>=8': False,
  'digit>=3': False,
  'hassec': False,
  'hashttp': False,
  'hasPfeat': False,
  'hasbhk': False,
  'hassq': False,
  'has/': False,
  'has@': False,
  'per': False,
  'BOS': True,
  '+1:word.lower()': '#edenpark',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'NN',
  '+1:postag[:2]': 'NN',
  '+1digit>=8': False,
  '+1digit>=3': False,
  '+1hassec': False,
  '+1hashttp': False,
  '+1hasPfeat': False,
  '+1hasbhk': False,
  '+1hassq': False,
  '+1has/': False,
  '+1has@': False,
  '+1per': False},
 {'bias': 1.0,
  'word.lower()': '#edenpark',
  'word[-3:]': 'ark',
  'word[-2:]': 'rk',
  'word[-4:]': 'Park',
  'word[:2]': '#E',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.

In [19]:
train_val=train_feat+val_feat
train_val_y=y_train+y_val
crf = sklearn_crfsuite.CRF(
    algorithm='pa',
#     c1=0.1,
#     c2=0.1,
    max_iterations=25,
    all_possible_transitions=True,
)


In [20]:
crf.fit(train_val, train_val_y)

CRF(algorithm='pa', all_possible_states=None, all_possible_transitions=True,
  averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
  calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=25,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [21]:
labels = list(crf.classes_)
print(labels)

['L', 'LA', 'A', 'O', 'T', 'N', 'P', 'C']


In [22]:
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [23]:
def post_process(x,y):
    for i in range(len(x)):
        sent=x[i]
#         print("Sent",sent)
        for j in range(len(sent)):
            word=sent[j]
            if 'http' in word.lower():
                y[i][j]='O'
#             elif greater(word,8):
#                 y[i][j]='T'          
                

In [24]:
def evaluate(crf,test_feat,y_test,clean=False,x=None):
    y_pred = crf.predict(test_feat)
    if(clean):
        post_process(x,y_pred)
        
    print(metrics.flat_f1_score(y_test, y_pred,average='weighted', labels=labels))
    sorted_labels = sorted(
        labels,
        key=lambda name: (name[1:], name[0])
    )
    print(metrics.flat_classification_report(
        y_test, y_pred, labels=sorted_labels, digits=3
    ))

In [25]:
evaluate(crf,test_feat,y_test)
# evaluate(crf,test_feat,y_test,True,x_test)

0.8917290217468125
              precision    recall  f1-score   support

           A      0.854     0.800     0.826     18761
           C      0.893     0.846     0.869      2046
           L      0.890     0.905     0.898     12756
           N      0.922     0.912     0.917      2571
           O      0.896     0.917     0.907     46181
           P      0.903     0.864     0.883      2859
           T      0.978     0.967     0.973      2024
          LA      0.924     0.948     0.936      7215

   micro avg      0.892     0.892     0.892     94413
   macro avg      0.908     0.895     0.901     94413
weighted avg      0.892     0.892     0.892     94413



In [26]:
evaluate(crf,train_val,train_val_y)

0.8917290217468125
              precision    recall  f1-score   support

           A      0.854     0.800     0.826     37522
           C      0.893     0.846     0.869      4092
           L      0.890     0.905     0.898     25512
           N      0.922     0.912     0.917      5142
           O      0.896     0.917     0.907     92362
           P      0.903     0.864     0.883      5718
           T      0.978     0.967     0.973      4048
          LA      0.924     0.948     0.936     14430

   micro avg      0.892     0.892     0.892    188826
   macro avg      0.908     0.895     0.901    188826
weighted avg      0.892     0.892     0.892    188826



In [27]:
eli5.show_weights(crf, top=5, show=['transition_features'])


From \ To,A,C,L,LA,N,O,P,T
A,0.492,-0.079,-0.029,-0.026,-0.129,0.057,0.015,-0.232
C,-0.16,0.716,-0.115,-0.1,-0.166,0.015,-0.205,-0.088
L,0.019,-0.142,0.586,-0.031,-0.046,0.039,-0.07,-0.251
LA,-0.011,-0.038,-0.158,0.526,-0.153,0.003,-0.11,0.064
N,-0.131,-0.197,-0.027,-0.088,0.463,-0.007,-0.325,0.021
O,0.072,-0.05,0.037,0.024,0.02,0.311,0.061,-0.05
P,-0.101,-0.082,-0.1,-0.172,-0.132,0.002,0.523,-0.156
T,-0.187,-0.125,-0.082,-0.144,0.022,-0.08,-0.146,0.636


In [28]:
eli5.show_weights(crf, top=20, targets=labels)

In [29]:
dicti={}
for l in labels:
    dicti[l]=[]
y_pred = crf.predict(test_feat)
for i in range(len(x_test)):
    for j in range(len(x_test[i])):
        if(y_pred[i][j]!=y_test[i][j]):
            dicti[y_test[i][j]].append((x_test[i][j],y_pred[i][j]))

In [30]:
lab='C'
print(len(dicti[lab]))
print(dicti[lab])

315
[('.', 'O'), ('.', 'O'), ('rs', 'O'), ('.', 'O'), ('5000', 'O'), ('.', 'O'), ('rs', 'O'), ('.', 'O'), ('7300/-', 'O'), ('rs', 'O'), ('.', 'O'), ('20000', 'O'), ('@16000', 'P'), ('13', 'O'), ('k', 'O'), ('PM', 'O'), ('5000', 'O'), ('4800', 'O'), ('/', 'O'), (',', 'O'), ('000', 'O'), ('@20k', 'P'), ('@36k', 'O'), ('4300', 'O'), ('Rs', 'P'), ('5700/-', 'P'), ('2590', 'LA'), ('sq', 'LA'), ('.', 'LA'), ('ft', 'LA'), ('2890', 'LA'), ('sq', 'LA'), ('.', 'LA'), ('ft', 'LA'), ('@4500', 'O'), ('@12', 'O'), ('@10', 'O'), ('@15', 'A'), ('@21', 'O'), ('@22', 'O'), ('@14', 'O'), ('all', 'O'), ('inclusive', 'O'), ('348Sq', 'LA'), ('.', 'LA'), ('ft', 'LA'), ('and', 'O'), ('Ground', 'A'), ('floor', 'A'), ('453Sq', 'LA'), ('.', 'LA'), ('ft', 'LA'), ('.', 'O'), ('.', 'O'), ('711sq', 'LA'), ('.', 'LA'), ('ft', 'LA'), ('@1', 'P'), (',', 'P'), ('10,000', 'P'), ('185', 'LA'), ('SQ', 'LA'), ('.', 'LA'), ('FT', 'LA'), ('.', 'O'), ('GROUND', 'A'), ('FLOOR', 'A'), ('@', 'O'), ('15', 'P'), ('Lac', 'P'), ('.',

In [3]:

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=None):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = unique_labels(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

#     fig, ax = plt.subplots()
#     im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
#     ax.figure.colorbar(im, ax=ax)
#     # We want to show all ticks...
#     ax.set(xticks=np.arange(cm.shape[1]),
#            yticks=np.arange(cm.shape[0]),
#            # ... and label them with the respective list entries
#            xticklabels=classes, yticklabels=classes,
#            title=title,
#            ylabel='True label',
#            xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
#     plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
#              rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i in range(cm.shape[0]):
#         for j in range(cm.shape[1]):
#             ax.text(j, i, format(cm[i, j], fmt),
#                     ha="center", va="center",
#                     color="white" if cm[i, j] > thresh else "black")
#     fig.tight_layout()
    return cm


In [4]:
plot_confusion_matrix(straight(y_test),straight(y_pred), classes=labels,normalize=True,title='Confusion matrix, without normalization')

NameError: name 'straight' is not defined

In [2]:
plot_confusion_matrix(straight(y_test),straight(y_pred), classes=labels,normalize=False,title='Confusion matrix, without normalization')

NameError: name 'straight' is not defined